In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

import os

from PIL import Image

from matplotlib.pyplot import imshow

Using TensorFlow backend.


In [2]:
def load_data():
        data = []
        paths = []
        for r, d, f in os.walk(r"D:\Downloads\flower-color-images\flower_images\flower_images"):
            for file in f:
                if '.png' in file:
                    paths.append(os.path.join(r, file))

        for path in paths:
            img = Image.open(path)
            x = np.array(img)
            x = x[...,:3]
            if(x.shape == (128,128,3)):
                data.append(x)

        x_train = np.array(data)
        x_train = x_train.reshape(len(data),128,128,3)
        return x_train

In [16]:
class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0001, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()
        generator = self.generator

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()
        model.add(Dense(512 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 512)))
        model.add(UpSampling2D())
        model.add(Conv2D(512, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(UpSampling2D())
        model.add(Conv2D(512, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(UpSampling2D())
        model.add(Conv2D(512, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(UpSampling2D())
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(256, kernel_size=(3,3), padding="same"))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.3))
        model.add(Conv2D(3, kernel_size=(2,2), padding="same"))
        model.add(LeakyReLU(alpha=0.3))
        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)
    

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        X_train = load_data()

        # Rescale -1 to 1
        X_train = X_train / 256

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 3, 3
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        gen_imgs = np.array(gen_imgs) * 256
        gen_imgs = gen_imgs.astype(int)

        fig=plt.figure(figsize=(20, 20))
        for i in range(1, c*r+1):
            img = gen_imgs[i-1]
            fig.add_subplot(r, c, i)
            plt.imshow(img)
        fig.savefig(r"C:\Users\Vee\Desktop\python\GAN\epoch_%d.png" % epoch)
        plt.close()
        self.generator.save("generator.h5")

In [17]:
dcgan = DCGAN()
dcgan.train(epochs=10001, batch_size=5, save_interval=100)

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_81 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
leaky_re_lu_39 (LeakyReLU)   (None, 64, 64, 32)        0         
_________________________________________________________________
dropout_78 (Dropout)         (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_82 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
zero_padding2d_9 (ZeroPaddin (None, 33, 33, 64)        0         
_________________________________________________________________
batch_normalization_65 (Batc (None, 33, 33, 64)        256       
_________________________________________________________________
leaky_re_lu_40 (LeakyReLU)   (None, 33, 33, 64)      

C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.549293, acc.: 40.00%] [G loss: 0.474721]


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
C:\Users\V

1 [D loss: 0.945787, acc.: 50.00%] [G loss: 0.609236]
2 [D loss: 0.439793, acc.: 80.00%] [G loss: 1.093102]
3 [D loss: 0.630943, acc.: 60.00%] [G loss: 0.521224]
4 [D loss: 0.543220, acc.: 70.00%] [G loss: 0.395911]
5 [D loss: 0.474590, acc.: 80.00%] [G loss: 0.751394]
6 [D loss: 0.430415, acc.: 80.00%] [G loss: 0.354738]
7 [D loss: 0.412186, acc.: 80.00%] [G loss: 0.707438]
8 [D loss: 0.351093, acc.: 90.00%] [G loss: 0.151993]
9 [D loss: 0.156982, acc.: 100.00%] [G loss: 0.523564]
10 [D loss: 0.140141, acc.: 100.00%] [G loss: 0.092526]
11 [D loss: 0.119138, acc.: 100.00%] [G loss: 0.283344]
12 [D loss: 0.167760, acc.: 100.00%] [G loss: 0.713291]
13 [D loss: 0.136671, acc.: 100.00%] [G loss: 0.207679]
14 [D loss: 0.457547, acc.: 80.00%] [G loss: 0.132369]
15 [D loss: 0.056501, acc.: 100.00%] [G loss: 0.198780]
16 [D loss: 0.148230, acc.: 100.00%] [G loss: 0.302073]
17 [D loss: 0.298858, acc.: 90.00%] [G loss: 0.311339]
18 [D loss: 0.106224, acc.: 100.00%] [G loss: 0.585533]
19 [D loss:

...

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


9800 [D loss: 0.956799, acc.: 60.00%] [G loss: 2.792890]
9801 [D loss: 0.117324, acc.: 100.00%] [G loss: 4.022177]
9802 [D loss: 0.048925, acc.: 100.00%] [G loss: 5.305243]
9803 [D loss: 0.996495, acc.: 50.00%] [G loss: 2.547543]
9804 [D loss: 1.201592, acc.: 20.00%] [G loss: 1.573069]
9805 [D loss: 0.190563, acc.: 100.00%] [G loss: 3.827209]
9806 [D loss: 0.951856, acc.: 40.00%] [G loss: 3.761283]
9807 [D loss: 1.213373, acc.: 50.00%] [G loss: 2.525890]
9808 [D loss: 0.341121, acc.: 90.00%] [G loss: 2.479522]
9809 [D loss: 0.461297, acc.: 60.00%] [G loss: 5.158562]
9810 [D loss: 0.300648, acc.: 100.00%] [G loss: 4.559632]
9811 [D loss: 0.963169, acc.: 40.00%] [G loss: 2.701534]
9812 [D loss: 0.312769, acc.: 90.00%] [G loss: 3.096301]
9813 [D loss: 0.658900, acc.: 80.00%] [G loss: 2.925107]
9814 [D loss: 0.786167, acc.: 50.00%] [G loss: 3.430422]
9815 [D loss: 0.190080, acc.: 100.00%] [G loss: 2.945168]
9816 [D loss: 0.978187, acc.: 40.00%] [G loss: 3.620348]
9817 [D loss: 0.396221, ac

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


9900 [D loss: 0.313063, acc.: 70.00%] [G loss: 3.407409]
9901 [D loss: 0.291007, acc.: 80.00%] [G loss: 2.668994]
9902 [D loss: 0.194337, acc.: 100.00%] [G loss: 3.310176]
9903 [D loss: 0.289788, acc.: 80.00%] [G loss: 2.447276]
9904 [D loss: 0.856970, acc.: 60.00%] [G loss: 3.048275]
9905 [D loss: 0.271066, acc.: 80.00%] [G loss: 3.411848]
9906 [D loss: 0.122577, acc.: 100.00%] [G loss: 3.683212]
9907 [D loss: 0.323686, acc.: 80.00%] [G loss: 2.421499]
9908 [D loss: 0.757053, acc.: 60.00%] [G loss: 3.411999]
9909 [D loss: 0.222653, acc.: 100.00%] [G loss: 3.391483]
9910 [D loss: 1.125949, acc.: 40.00%] [G loss: 1.984572]
9911 [D loss: 0.409981, acc.: 80.00%] [G loss: 2.270326]
9912 [D loss: 0.868542, acc.: 60.00%] [G loss: 2.312410]
9913 [D loss: 0.322009, acc.: 70.00%] [G loss: 3.727318]
9914 [D loss: 0.193976, acc.: 100.00%] [G loss: 4.403842]
9915 [D loss: 0.398532, acc.: 80.00%] [G loss: 3.838264]
9916 [D loss: 0.970809, acc.: 50.00%] [G loss: 3.852376]
9917 [D loss: 0.309107, acc

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
